<center><h1>VC08: Aprendizaje semi-supervisado</h1></center>

En esta práctica aprenderemos a llevar a cabo un aprendizaje semi-supervisado. En concreto, estudiaremos el funcionamiento y la utilización del popular algoritmo de EM para aprender un modelo naive Bayes en semi-supervisado, así como la estrategia de aprendizaje conocida como "co-training".

Empezaremos por programar una implementación del clasificador que usaremos, el clásico Naive Bayes. Este clasificador probabilístico guarda los parámetros relativos a la distribución de probabilidad marginal de clase ($\{p(C=c)\}_{c=1}^{|C|}$) y las distribuciones de probabilidad condicional de las variables predictoras dada la clase ($\{p(X_i=x_i|C=c)\}_{x_i=1}^{|X_i|}$ para los diferentes valores de la variable clase $C=c$ y para todas las variables predictoras). Con estos parámetros se puede calcular la probabilidad de un caso, $\mathbf{x}$:
$$  p(C=c|\mathbf{x})\propto p(C=c)\prod_{i=1}^{v} p(x_i|C=c)$$
normalizándolos para todos los posibles valores de $c$ (todas las clases posibles) y 
con $v$ siendo el número de variables predictoras 
(2 en este caso) para que la 
suma sea $\sum_{c=1}^{|C|} p(C=c|\mathbf{x})=1$.

Por conveniencia, también guardaremos otras variables como la cardinalidad de las diferentes variables y el índice de la variable clase.


In [ ]:
import numpy as np

class naiveBayes:
    def __init__(self, iClass, cardinalities):
        self.iClass = iClass
        self.cardinalities = cardinalities.copy()
        self.Pc = np.zeros(cardinalities[iClass])
        self.Pxc = []
        for i in np.arange(len(self.cardinalities)):
            aux = np.array([])
            if i != iClass:
                aux = np.zeros((cardinalities[i],cardinalities[iClass]))
            self.Pxc.append(aux)    

Una vez diseñado, vamos a lo importante. Un clasificador ha de 
poder ser aprendido y usado para predecir. Por ello, diseñaremos 
una función para cada procedimiento. 
Para predecir, se han de calcular las siguiente probabilidades:
$$ p(C=c)\prod_{i=1}^{v} p(x_i|C=c)$$
para todos los posibles valores de $c$ (todas las clases posibles) y 
con $v$ siendo el número de variables predictoras 
(2 en este caso). Finalmente, normalizamos para que la 
suma de $\sum_{c=1}^{|C|} p(C=c|x)=1$.
La función de predicción 
sería así:

In [ ]:
def predictNB(model, instance):
    probs = model.Pc.copy()
    for i in np.arange(len(model.cardinalities)):
        if i != model.iClass:
            probs *= model.Pxc[i][#### TU CODIGO AQUI ####]

    return probs/np.sum(probs)


Considerando la estructura definida para guardar los 
parámetros y lo visto en clase sobre aprendizaje de 
parámetros de máxima verosimilitud en un entorno de 
aprendizaje semi-supervisado, diseñaremos la función 
de aprendizaje:


In [ ]:
def learnNB(L, U, pesosU, iClass, cardinalities, smoothing=1):
    modelo = naiveBayes(iClass, cardinalities)
    
    # Aprender de casos etiquetados
    for i in np.arange(L.shape[0]):
        modelo.Pc[L[i,iClass]] += 1
        
        for j in np.arange(len(cardinalities)):
            if j != iClass:
                modelo.Pxc[j][L[i,j],L[i,iClass]] += #### TU CODIGO AQUI ####

    for u in np.arange(U.shape[0]):
        modelo.Pc += pesosU[u,:]
        
        for j in np.arange(len(cardinalities)):
            if j != iClass:
                modelo.Pxc[j][U[u,j],:] += #### TU CODIGO AQUI ####
    
    modelo.Pc += smoothing # Laplace smoothing
    modelo.Pc /= np.sum(modelo.Pc)
    for j in np.arange(len(cardinalities)):
        if j != iClass:
            modelo.Pxc[j] += smoothing # Laplace smoothing
            modelo.Pxc[j] /= np.sum(modelo.Pxc[j],0)
    
    return modelo


Con esto completamos el diseño del clasificador Naive Bayes para el entorno semi-supervisado.

Ahora ya podemos crear el algoritmo EM, que itera los pasos E y M de forma sencilla hasta que converge:


In [ ]:
def EM(L,U, iClass, cardinalities, epsilon=0.001):
    modelo, pesosU = inicializar(L, U, iClass, cardinalities)

    convergencia = False
    it = 0
    while not convergencia:
        it += 1
        print('Iteracion', it)
        pesosU = EStep(L, U, pesosU, modelo)

        antModelo = modelo
        
        modelo = MStep(L, U, pesosU, iClass, cardinalities, antModelo)
    
        convergencia = testConvergencia(modelo, antModelo, epsilon)
    
    return modelo, pesosU

Faltarían por programar las tres funciones fundamentales 
del EM: el paso E (<b>EStep</b>),  el paso M (<b>MStep</b>) 
y la función que comprueba la convergencia (<b>testConvergencia</b>). 
Además, falta la inicialización donde asignamos valores iniciales 
a los pesos de las instancias no etiquetadas para aprender la 
primera versión del modelo. En concreto, en esta ocasión 
asignaremos a todos los casos la misma probabilidad de 
pertenecer a cualquier clase:


In [ ]:
def inicializar(L, U, iClass, cardinalities):
    pesosU = np.ones((U.shape[0],cardinalities[iClass]))
    pesosU /= cardinalities[iClass]
    
    modelo = learnNB(L, U, pesosU, iClass, cardinalities)

    return modelo, pesosU


El paso E consiste en recalcular los pesos dado un modelo y el paso M en aprender una nueva versión del modelo:
    

In [ ]:
def EStep(L, U, pesosU, modelo):
    nPesosU = np.zeros(pesosU.shape)
    for u in np.arange(U.shape[0]):
        nPesosU[u,:] = #### TU CODIGO AQUI ####

    return nPesosU

def MStep(L, U, pesosU, iClass, cardinalities, antModelo):
    return #### TU CODIGO AQUI ####


Mediremos la convergencia atendiendo al criterio de si 
la distancia euclídea entre los parámetros del paso anterior 
y del actual es menor que <i>epsilon</i> o no:


In [ ]:
def testConvergencia(modeloA, modeloB, epsilon=0.001):
    resultado = np.sum((modeloA.Pc-modeloB.Pc)**2)

    for j in np.arange(len(modeloA.cardinalities)):
        if j != modeloA.iClass:
            resultado += np.sum((modeloA.Pxc[j]-modeloB.Pxc[j])**2)
  
    resultado = #### TU CODIGO AQUI ####
    
    return resultado < epsilon


Una vez hemos diseñado completamente el algoritmo EM y el clasificador NB, podemos proceder a su uso. 
Creamos un dataset y hacemos la llamada al EM:


In [ ]:
from sklearn.datasets import make_classification

np.random.seed(23)
nPredVars = 10
iClass = nPredVars
nSampleXsubset = 20
cardinalities = np.repeat(2,nPredVars+1)

# Simulamos un dataset
X,y = make_classification(n_samples=nSampleXsubset*3,n_features=nPredVars,n_redundant=0)
X[X<0] = 0;X[X>0] = 1 # discretizamos las variables predictoras
X = X.astype(int)
y.shape=(len(y),1)

L = np.concatenate((X[:nSampleXsubset,:],
                    y[:nSampleXsubset,:]),axis=1)
U = np.concatenate((X[nSampleXsubset:(nSampleXsubset*2),:],
                    y[nSampleXsubset:(nSampleXsubset*2),:]),axis=1)
U[:,iClass] = np.nan

modelo, pesos = EM(L, U, iClass, cardinalities, 0.001)
print('Peso: ',pesos[1,0], ' (** ESTE ES EL RESULTADO A INCLUIR EN EL CAMPUS**)')



Como en anteriores ocasiones, podemos estudiar la bondad del agrupamiento ya que se conoce la realidad:


In [ ]:
test = np.concatenate((X[nSampleXsubset*2:,:],
                    y[nSampleXsubset*2:,:]),axis=1)

realLabels = test[:,iClass]
predLabels = np.zeros(realLabels.shape)
for i in np.arange(test.shape[0]):
    probs = predictNB(modelo, test[i,:])
    predLabels[i] = np.argmax(probs)

In [ ]:
def matriz_confusion(cat_real, cat_pred):
    cats = np.unique(cat_real)
    clusts = np.unique(cat_pred)
    mat = np.array([[np.sum(np.logical_and(cat_real==cats[i], cat_pred==clusts[j])) 
                     for j in np.arange(clusts.size)] 
                    for i in np.arange(cats.size)])
    return(mat)

def medida_error(mat):
    tot = np.sum(mat)
    aux = mat.copy()
    np.fill_diagonal(aux, 0)
    return float(np.sum(aux))/tot

def medida_precision(mat, l, k):
    return mat[l,k]/float(np.sum(mat[:,k]))

def medida_recall(mat, l, k):
    return mat[l,k]/float(np.sum(mat[l,:]))

def medida_f1_especifica(mat, l, k):
    prec = medida_precision(mat, l, k)
    rec = medida_recall(mat, l, k)
    if (prec+rec)==0:
        return 0
    else:
        return 2*prec*rec/(prec+rec)

def medida_f1(mat):
    return medida_f1_especifica(mat, 1, 1)

mC = matriz_confusion(realLabels,predLabels)

print(mC)
print('El error del clasificador es = ', medida_error(mC))
print('El valor F1 del clasificador es = ', medida_f1(mC))

<hr>
<h2>Implementaciones en librerías de Python</h2>

La librería ScikitLearn implementa diversas versiones del clasificador NB. En el caso de <b>MultinomialNB</b>, la distribución de probabilidad que se modela es diferente: una distribución de probabilidad multinomial donde el valor de cada variable predictora es el conteo de esta distribución de probabilidad:

\begin{aligned}
\Pr(X_{1}=x_{1},\dots, X_{v}=x_{v})&{}={\begin{cases}{\displaystyle {n! \over x_{1}!\cdot\dots\cdot x_{k}!}p_{1}^{x_{1}}\cdot \dots \cdot p_{v}^{x_{v}}},\quad &{\text{when }}\sum _{i=1}^{k}x_{i}=n\\\\0&{\text{otherwise,}}\end{cases}}
\end{aligned}
para cualquier conjunto $\{x_1, \dots, x_v\}$ de valores no negativos.

Si quisiésemos incluir esta versión del NB en el algoritmo EM, deberíamos modificar las funciones específicas que hacen llamadas al modelo pero no la general iterativa del EM.

En primer lugar, deberíamos preparar una función para el aprendizaje del modelo cuando los datos son semi-supervisados:


In [ ]:
from sklearn.naive_bayes import MultinomialNB

def sklearnNB(L, U, pesosU, iClass, cardinalities):
    modelo = MultinomialNB()

    modelo.partial_fit(np.delete(L, iClass, axis=1),
                       L[:,iClass],
                       classes=[0,1])

    for c in np.arange(cardinalities[iClass]):
        impU = U.copy()
        impU[:,iClass] = c
        modelo.partial_fit(np.delete(impU, iClass, axis=1),
                           impU[:,iClass],
                           sample_weight = pesosU[:,c])

    return modelo


La función de predicción del NB, en este caso, ya está integrada en la definición de la clase de Scikit-learn.

Las cuatro funciones auxiliares del EM se deben adecuar a este nuevo modelo: 


In [ ]:
def inicializar(L, U, iClass, cardinalities):
    pesosU = np.ones((U.shape[0],cardinalities[iClass]))
    pesosU /= cardinalities[iClass]
    
    modelo = sklearnNB(L, U, pesosU, iClass, cardinalities)

    return modelo, pesosU

def EStep(L, U, pesosU, modelo):

    nPesosU = modelo.predict_proba(np.delete(U, iClass, axis=1))

    return nPesosU

def MStep(L, U, pesosU, iClass, cardinalities, antModelo):
    modelo = sklearnNB(L, U, pesosU, iClass, cardinalities)
    return modelo

def testConvergencia(modeloA, modeloB, epsilon=0.001):
    resultado = np.sum((np.exp(modeloA.class_log_prior_) - 
                        np.exp(modeloB.class_log_prior_))**2)
    resultado += np.sum((np.exp(modeloA.feature_log_prob_) - 
                         np.exp(modeloB.feature_log_prob_))**2)

    resultado = np.sqrt(resultado)
    
    return resultado < epsilon


Una vez hemos diseñado completamente las distintas funciones del algoritmo EM y esta nueva versión del NB, podemos proceder a su uso. 
Creamos un dataset y hacemos la llamada al EM:


In [ ]:
np.random.seed(23)
nPredVars = 10
iClass = nPredVars
nSampleXsubset = 20
cardinalities = np.repeat(2,nPredVars+1)

# Simulamos un dataset
X,y = make_classification(n_samples=nSampleXsubset*3,n_features=nPredVars,n_redundant=0)
X[X<0] = 0;X[X>0] = 1 # discretizamos las variables predictoras
X = X.astype(int)
y.shape=(len(y),1)

L = np.concatenate((X[:nSampleXsubset,:],
                    y[:nSampleXsubset,:]),axis=1)
U = np.concatenate((X[nSampleXsubset:(nSampleXsubset*2),:],
                    y[nSampleXsubset:(nSampleXsubset*2),:]),axis=1)
U[:,iClass] = np.nan

skmodelo, skpesos = EM(L, U, iClass, cardinalities, 0.001)
print(skpesos)

In [ ]:
test = np.concatenate((X[nSampleXsubset*2:,:],
                       y[nSampleXsubset*2:,:]),axis=1)

skRealLabels = test[:,iClass]
skPredLabels = np.zeros(realLabels.shape)
test = np.delete(test,iClass,1)
skPredLabels = np.argmax(skmodelo.predict_proba(test), axis=1)

In [ ]:
mC = matriz_confusion(skRealLabels,skPredLabels)

print(mC)
print('El error del clasificador es = ', medida_error(mC))
print('El valor F1 del clasificador es = ', medida_f1(mC))


El algoritmo EM, ejecutado en una única ocasión, puede dar un resultado no óptimo: se está eligiendo una inicialización que lleva al algoritmo a encallar en un óptimo local. Si se ejecuta en varias ocaciones, eventualmente se obtendrá el resultado óptimo.

Podríamos comparar el resultado de nuestro algoritmo y el de la implementación de ScikitLearn para observar si devuelven el mismo resultado:
    

In [ ]:
# Si comparamos el resultado de ambos algoritmos, el nuestro y el de ScikitLearn
mC_comp = matriz_confusion(predLabels,skPredLabels)

print('Matriz de confusión:')
print(mC_comp)
print('El valor del error (diferencia) es = ', medida_error(mC_comp))

<hr />
<center><h1>Co-agrupamiento</h1></center>

Si disponemos de dos conjuntos de variables predictoras (disjuntos y, en la medida de lo posible, independientes), se puede alternar el aprendizaje semi-supervisado de uno y otro subconjunto con el objetivo de localizar, mediante uno u otro procedimiento, de algún caso con alta probabilidad de pertenecer a cierta clase. Esta idea la podemos expresar de la siguiente manera. 

En primer lugar, generaremos los dos conjuntos de variables predictoras:


In [ ]:
np.random.seed(23)
nPredVars = 20
iClass = nPredVars
nSampleXsubset = 20
cardinalities = np.repeat(2,nPredVars+1)


# Simulamos un dataset
X,y = make_classification(n_samples=nSampleXsubset*2,n_features=nPredVars)
X[X<0] = 0;X[X>0] = 1
X = X.astype(int)
y.shape=(len(y),1)

print(X[np.ix_(np.arange(nSampleXsubset),np.arange(nPredVars//2,nPredVars))].shape)
print((y[np.arange(nSampleXsubset)].T).shape)
L_a = np.concatenate((X[np.ix_(np.arange(nSampleXsubset),np.arange(nPredVars//2,nPredVars))],
                      y[np.arange(nSampleXsubset),:]),axis=1)
L_b = np.concatenate((X[np.ix_(np.arange(nSampleXsubset),np.arange(nPredVars//2))],
                      y[np.arange(nSampleXsubset)]),axis=1)
U_a = np.concatenate((X[np.ix_(np.arange(nSampleXsubset,2*nSampleXsubset),np.arange(nPredVars//2,nPredVars))],
                      y[np.arange(nSampleXsubset,2*nSampleXsubset),:]),axis=1)
U_b = np.concatenate((X[np.ix_(np.arange(nSampleXsubset,2*nSampleXsubset),np.arange(nPredVars//2))],
                      y[np.arange(nSampleXsubset,2*nSampleXsubset),:]),axis=1)

cardinalities_a = cardinalities[np.arange(nPredVars//2,nPredVars)]
cardinalities_b = cardinalities[np.arange(nPredVars//2)]
iClass = nPredVars//2
U_a[:,iClass] = np.nan
U_b[:,iClass] = np.nan


Así, los casos etiquetados y los no etiquetados están expresados por medio de los dos conjuntos de variables.

Si fijamos un umbral <it>th</it> de ejemplos que consideraremos etiquetados si superan ese umbral de probabilidad, el proceso consiste en iterar los dos siguientes subprocesos:

- Aprender con las variables del conjunto $A$, localizar posibles ejemplos (positivos y negativos) con alta probabilidad y, en su caso, pasarlos al conjunto de etiquetados.

- Aprender con las variables del conjunto $B$, localizar posibles ejemplos (positivos y negativos) con alta probabilidad y, en su caso, pasarlos al conjunto de etiquetados.



In [ ]:
th = 0.95

seguir = True
while seguir :
    ini = L_a.shape[0]
    modelo_a, pesos_a = EM(L_a, U_a, iClass, cardinalities, 0.001)
    print(pesos_a)
    toLabelNeg = np.where(pesos_a[:,0]>th)[0]
    if len(toLabelNeg)>0:
        print('Asignar a negativos: ', toLabelNeg)
        aux = U_a[toLabelNeg,:]
        aux[:,iClass] = 0
        L_a = np.concatenate((L_a,aux),axis=0)
        aux = U_b[toLabelNeg,:]
        aux[:,iClass] = 0
        L_b = np.concatenate((L_b,aux),axis=0)
        U_a = np.delete(U_a, toLabelNeg, axis=0)
        U_b = np.delete(U_b, toLabelNeg, axis=0)
        pesos_a = np.delete(pesos_a, toLabelNeg, axis=0)

    toLabelPos = np.where(pesos_a[:,1]>th)[0]
    if len(toLabelPos)>0:
        print('Asignar a positivos: ', toLabelPos)
        aux = U_a[toLabelPos,:]
        aux[:,iClass] = 1
        L_a = np.concatenate((L_a,aux),axis=0)
        aux = U_b[toLabelPos,:]
        aux[:,iClass] = 1
        L_b = np.concatenate((L_b,aux),axis=0)
        U_a = np.delete(U_a, toLabelPos, axis=0)
        U_b = np.delete(U_b, toLabelPos, axis=0)
        pesos_a = np.delete(pesos_a, toLabelPos, axis=0)

    modelo_b, pesos_b = EM(L_b, U_b, iClass, cardinalities, 0.001)
    print(pesos_b)

    toLabelNeg = np.where(pesos_b[:,0]>th)[0]
    if len(toLabelNeg)>0:
        print('Asignar a negativos: ', toLabelNeg)
        aux = U_a[toLabelNeg,:]
        aux[:,iClass] = 0
        L_a = np.concatenate((L_a,aux),axis=0)
        aux = U_b[toLabelNeg,:]
        aux[:,iClass] = 0
        L_b = np.concatenate((L_b,aux),axis=0)
        U_a = np.delete(U_a, toLabelNeg, axis=0)
        U_b = np.delete(U_b, toLabelNeg, axis=0)
        pesos_b = np.delete(pesos_b, toLabelNeg, axis=0)

    toLabelPos = np.where(pesos_b[:,1]>th)[0]
    if len(toLabelPos)>0:
        print('Asignar a positivos: ', toLabelPos)
        aux = U_a[toLabelPos,:]
        aux[:,iClass] = 1
        L_a = np.concatenate((L_a,aux),axis=0)
        aux = U_b[toLabelPos,:]
        aux[:,iClass] = 1
        L_b = np.concatenate((L_b,aux),axis=0)
        U_a = np.delete(U_a, toLabelPos, axis=0)
        U_b = np.delete(U_b, toLabelPos, axis=0)
        pesos_b = np.delete(pesos_b, toLabelPos, axis=0)

    seguir = (L_a.shape[0] - ini) > 0


Para finalizar, y como un simple ejercicio de curiosidad, podemos ver cuántos ejemplos fueron considerados en algún momento del proceso como certero (o con suficiente certeza) y pasaron al grupo de etiquetados:


In [ ]:
print(L_a.shape[0]-nSampleXsubset, 'instancias se incluyeron en el dataset de etiquetados')